# Analysis of the results

In [1]:
import os
import json
import numpy as np
from scipy import stats

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


## Get best results for each fold

In [2]:
def construct_path_from_parent(current_path, subdir_list):
    # Get the parent directory
    parent_dir = os.path.dirname(current_path)
    
    # Construct the path using os.path.join
    return os.path.join(parent_dir, *subdir_list)

In [3]:
current_dir = os.getcwd()
dataset_name = "kvasir"
model_name = "UNet_vanilla"

folds = [f'fold_{i}' for i in range(5)]
criteria = 'loss_dice'
perf_metrics = ['dice_score', 'IoU_score', 'precision', 'recall', 'accuracy']
result_dict: dict = {}

In [4]:
def get_best_metrics(summary_loc: str, fold_nb: int, result_dict: dict, 
                     perf_metrics: list, criteria: str) -> None:
    with open(summary_loc) as file:
        data = json.load(file)
        
    data = data['test']

    best_idx = np.argmin(data[criteria])
    
    fold_dict: dict = {}
    for metric in perf_metrics:
        fold_dict[metric] = data[metric][best_idx]
    fold_dict['epoch'] = best_idx
    
    if fold_nb in result_dict:
        raise KeyError(f"Key '{fold_nb}' already present in dictionary.")
    
    result_dict[fold_nb] = fold_dict

In [5]:
def build_path(dataset_name: str, model_name: str, fold_nb: str) -> str:
    subdirs = ["training", "outputs", dataset_name, model_name, fold_nb, "summary.json"]

    summary_loc = construct_path_from_parent(current_dir, subdirs)
    if os.path.exists(summary_loc):
        print(f'summary.json loc: {summary_loc}')
    else:
        print(f'No file at the path you provided.')
    
    return summary_loc

In [7]:
for fold in folds:
    summary_loc = build_path(dataset_name, model_name, fold)
    get_best_metrics(summary_loc, fold, result_dict, perf_metrics, criteria)

summary.json loc: /home/rob/Documents/Github/seg-equi-architectures/src/Benchmarks/training/outputs/kvasir/UNet_vanilla/fold_0/summary.json
summary.json loc: /home/rob/Documents/Github/seg-equi-architectures/src/Benchmarks/training/outputs/kvasir/UNet_vanilla/fold_1/summary.json
summary.json loc: /home/rob/Documents/Github/seg-equi-architectures/src/Benchmarks/training/outputs/kvasir/UNet_vanilla/fold_2/summary.json
summary.json loc: /home/rob/Documents/Github/seg-equi-architectures/src/Benchmarks/training/outputs/kvasir/UNet_vanilla/fold_3/summary.json
summary.json loc: /home/rob/Documents/Github/seg-equi-architectures/src/Benchmarks/training/outputs/kvasir/UNet_vanilla/fold_4/summary.json


{'fold_0': {'dice_score': 0.8734993636608124,
  'IoU_score': 0.7769129276275635,
  'precision': 0.9252771437168121,
  'recall': 0.8713351935148239,
  'accuracy': 0.9695890247821808,
  'epoch': 179},
 'fold_1': {'dice_score': 0.8739893138408661,
  'IoU_score': 0.7807178944349289,
  'precision': 0.9170181155204773,
  'recall': 0.8714165687561035,
  'accuracy': 0.9695335924625397,
  'epoch': 189},
 'fold_2': {'dice_score': 0.8759756088256836,
  'IoU_score': 0.7815407812595367,
  'precision': 0.9347810447216034,
  'recall': 0.8629176914691925,
  'accuracy': 0.9699866771697998,
  'epoch': 195},
 'fold_3': {'dice_score': 0.8736938834190369,
  'IoU_score': 0.7777127772569656,
  'precision': 0.8957230746746063,
  'recall': 0.898811087012291,
  'accuracy': 0.9737357646226883,
  'epoch': 183},
 'fold_4': {'dice_score': 0.8492033183574677,
  'IoU_score': 0.7394035309553146,
  'precision': 0.8747237622737885,
  'recall': 0.8647194355726242,
  'accuracy': 0.9624717682600021,
  'epoch': 193}}

## Get average with 95% CI

In [8]:
results = {}

for metric in perf_metrics:
    data = np.array([result_dict[f"fold_{i}"][metric] for i in range(5)])
    mean = np.mean(data)
    ci = stats.t.interval(alpha=0.95, df=len(data)-1, loc=np.mean(data), scale=stats.sem(data))
    results[metric] = {
        "mean": mean,
        "95% CI": ci
    }

## Create output file

In [9]:
output_path = '/home/rob/Documents/3_projects/bench/analysis'
with open(f'{output_path}/data.json', 'w') as fp:
    json.dump(results, fp, default=str, indent=4)